# COGS 108 - Data Checkpoint

# Names

- Aasem Fituri
- Casey Hild
- Carlos van der Ley
- Jeremy Quinto

<a id='research_question'></a>
# Research Question

Can we find a correlation between cancer rates and socioeconomic status?  More specifically, how does income, education, and employment status affect cancer rates across the United States?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [18]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Reading in the dataset
data_cancer = pd.read_csv('data/cancer_reg.csv')
data_cancer.head()

,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
0,0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657


# Data Cleaning

In [32]:
# Adding County, State columns from info in the 'geography' column
data_cancer[['County', 'State']] = data_cancer['geography'].str.split(',', n=1, expand=True).astype('string')
data_cancer['State'] = data_cancer['State'].str.replace(' ', '')
data_cancer.head()

,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,County,State,Region
0,0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",...,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831,Kitsap County,Washington,N/A
1,1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",...,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096,Kittitas County,Washington,N/A
2,2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",...,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488,Klickitat County,Washington,N/A
3,3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",...,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841,Lewis County,Washington,N/A
4,4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",...,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657,Lincoln County,Washington,N/A


In [35]:
# Assigning a Region to each County based on its state
West = ['California', 'Nevada']
NorthWest = ['Washington', 'Oregon', 'Idaho', 'Montana', 'Wyoming']
SouthWest = ['Utah', 'Colorado', 'Arizona', 'New Mexico', 'Texas', 'Oklahoma']
MidWest = ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri', 'Wisconsin', 'Illinois', 'Indiana', 'Michigan', 'Indiana', 'Kentucky', 'Ohio']
SouthEast = ['Arkansas', 'Louisiana', 'Mississippi', 'Tennessee', 'North Carolina', 'South Carolina', 'Georgia', 'Alabama', 'Florida']
MidAtlantic = ['Virginia', 'West Virginia', 'Delaware', 'Maryland', 'Pennsylvania', 'New Jersey', 'Connecticut', 'New York']
NorthEast = ['Connecticut', 'Rhode Island', 'Massachusetts', 'Vermont', 'New Hampshire', 'Maine']
# Length of all lists combines to 50
print(len(West) + len(NorthWest) + len(SouthWest) + len(MidWest) + len(SouthEast) + len(MidAtlantic) + len(NorthEast))

# Create the 'Region' column
data_cancer['Region'] = data_cancer['State'].apply(lambda x: 'West' if x in West else
                                                         'NorthWest' if x in NorthWest else
                                                         'SouthWest' if x in SouthWest else
                                                         'MidWest' if x in MidWest else
                                                         'SouthEast' if x in SouthEast else
                                                         'MidAtlantic' if x in MidAtlantic else
                                                         'NorthEast' if x in NorthEast else '')

pd.set_option('display.max_rows', 10)
data_cancer.groupby('Region').count()

50


,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,County,State
Region,,,,,,,,,,,,,,,,,,,,,
,463,463,463,463,463,463,463,463,463,463,...,463,463,463,463,463,463,463,463,463,463
MidAtlantic,227,227,227,227,227,227,227,227,227,227,...,227,227,227,227,227,227,227,227,227,227
MidWest,1039,1039,1039,1039,1039,1039,1039,1039,1039,1039,...,1039,1039,1039,1039,1039,1039,1039,1039,1039,1039
NorthEast,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
NorthWest,188,188,188,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188
SouthEast,600,600,600,600,600,600,600,600,600,600,...,600,600,600,600,600,600,600,600,600,600
SouthWest,412,412,412,412,412,412,412,412,412,412,...,412,412,412,412,412,412,412,412,412,412
West,74,74,74,74,74,74,74,74,74,74,...,74,74,74,74,74,74,74,74,74,74


Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION